In [1]:
%load_ext autoreload
%autoreload 2

In [173]:
import numpy as np
import proj_funcs as pr
import pandas as pd
from constructor import Constructor
from optimiser import cost_func,gen_U_from_param_array,fidelity
import random
from scipy.optimize import minimize
from scipy.stats import unitary_group

In [15]:
dim = 2

gm_matrices = pr.get_gellmann_matrices(dim)

bases = pr.get_bases_from_matrices(gm_matrices)
evs = pr.get_eigenvalues_from_matrices(gm_matrices)


In [81]:

basis = np.array(bases[2])
print(evs[2])
print(pd.DataFrame(basis))

proj = pr.construct_projector(basis)
print(pd.DataFrame(proj))

output = proj @ basis

print(output)



[1.0, -1.0]
          0         1
0  0.707107  0.707107
1 -0.707107  0.707107
                    0                   1
0  0.707107+0.000000j  0.707107+0.000000j
1 -0.707107+0.000000j  0.707107+0.000000j
[[-2.23711432e-17+0.j  1.00000000e+00+0.j]
 [-1.00000000e+00+0.j  2.23711432e-17+0.j]]


In [90]:
eta = 1/2
phase = np.pi
BS = ct.BeamSplitter(1,4,eta)
PS = ct.PhaseShifter(1,4,phase)
# print
print(BS.global_U())
print(PS.global_U())

(array([[0],
       [1]]), array([[0, 1]]))
[[ 0.70710678+0.j  0.70710678+0.j  0.        +0.j  0.        +0.j]
 [ 0.70710678+0.j -0.70710678+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  1.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1.        +0.j]]
(array([[0],
       [1]]), array([[0, 1]]))
[[-1.+1.2246468e-16j  0.+0.0000000e+00j  0.+0.0000000e+00j
   0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  1.+0.0000000e+00j  0.+0.0000000e+00j
   0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  0.+0.0000000e+00j  1.+0.0000000e+00j
   0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  0.+0.0000000e+00j  0.+0.0000000e+00j
   1.+0.0000000e+00j]]


# Test constructor class

To test the constructor class and optimiser we will construct a 4-D clements scheme and use an optimiser to find phases.

In [170]:
dim = 4
U = np.eye(dim)
U=unitary_group.rvs(dim)

pattern = [0,2,1,0,2,1]

phase_guess = [2*np.pi*random.uniform(0,1) for i in range(dim)]

main_guess = [np.pi*random.uniform(0,1) if i%2 else 2*np.pi*random.uniform(0,1) for i in range(2*len(pattern))]

initial_guess = phase_guess + main_guess
print(initial_guess)
args = (dim,U,pattern,None)


res = minimize(cost_func,x0=initial_guess,args =args,method='BFGS')



[6.1633911445272025, 6.173478134500623, 0.530324510735575, 1.3398742956075111, 1.1602477049574473, 1.6847155903888906, 0.00028749151186628126, 0.04980491568452529, 2.0127190497571057, 3.1291082260806724, 5.535195408561557, 1.8306920601223409, 2.635617035895119, 2.4205559389083944, 4.978757173599384, 1.2231614579711263]


In [177]:
'''Test many random matrices'''

dim = 6
pattern = [0,2,1,0,2,1] #  4 mode ideal clements
pattern = [0,2,4,3,1,0,2,4,3,1,0,2,4,3,1] #  6 mode ideal clements

num_steps = 10

cf_array = []
fid_array = []

for i in range(num_steps):

    print(f'{i+1}/{num_steps}',end = '\r')

    phase_guess = [2*np.pi*random.uniform(0,1) for i in range(dim)]

    main_guess = [np.pi*random.uniform(0,1) if i%2 else 2*np.pi*random.uniform(0,1) for i in range(2*len(pattern))]

    initial_guess = phase_guess + main_guess

    U=unitary_group.rvs(dim)

    args = (dim,U,pattern,None)

    res = minimize(cost_func,x0=initial_guess,args =args,method='BFGS')

    cf = cost_func(res.x,*args)

    U_opt = gen_U_from_param_array(res.x,dim,pattern,None)

    fid = fidelity(U_opt,U)

    fid_array.append(fid)

    cf_array.append(cf)

print(f'Fidelity = {np.mean(fid_array)} ± {np.std(fid_array)}')
print(f'Cost func = {np.mean(cf_array)} ± {np.std(cf_array)}')



Fidelity = 0.9999999999743275 ± 1.5561688251576005e-11
Cost func = 3.0806919886258696e-10 ± 1.8674042873580058e-10


True